# Parte 2: Desarrollo de la Red Neuronal

### Forward Propagation

In [8]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# cargo el conjunto de datos
all_data = pd.read_csv("pasajeros_actualizado.csv")

# selecciono todas las columnas de entrada del conjunto de datos 
all_inputs = all_data.iloc[:, 0:16].values

# selecciono la columna de salida del conjunto de datos
all_outputs = all_data.iloc[:, -1].values

# escalo los datos de entrada (StandarScaler realiza una estandarización de los datos)
scaler = StandardScaler()
# transformo las entradas para tener una media de 0 y una desviación estándar de 1
all_inputs = scaler.fit_transform(all_inputs)

# Dividir los conjuntos de datos de entrenamiento y prueba
X_train, X_test, Y_train, Y_test = train_test_split(all_inputs, all_outputs, test_size=1 / 3)
n = X_train.shape[0] # cantidad de ejemplos en el conjunto de entrenamiento (n = 17317)

# defino funciones de activación
relu = lambda x: np.maximum(x, 0)  # ReLU porque no tengo valores negativos
logistic = lambda x: 1 / (1 + np.exp(-x)) # Logística para la capa de salida (salida binaria) 

# inicializo pesos y sesgos de la red con una semilla = 79 (para reproducibilidad)
np.random.seed(79)
w_hidden = np.random.rand(12, 16)   # pesos primera capa oculta
w_hidden2 = np.random.rand(8, 12)   # pesos segunda capa oculta        
w_output = np.random.rand(1, 8)     # pesos capa de salida

b_hidden = np.random.rand(12, 1)    # sesgos primera capa oculta   
b_hidden2 = np.random.rand(8, 1)    # sesgos segunda capa oculta
b_output = np.random.rand(1, 1)     # sesgos capa de salida

# funcion Forward Propagation 
def forward_prop(X):
    Z1 = w_hidden @ X + b_hidden       # entrada ponderada de la primera capa oculta
    A1 = relu(Z1)                      # ReLU en la primera capa oculta
    Z2 = w_hidden2 @ A1 + b_hidden2    # entrada ponderada de la segunda capa oculta
    A2 = relu(Z2)                      # ReLU en la segunda capa oculta
    Z3 = w_output @ A2 + b_output      # entrada ponderada de la capa de salida
    A3 = logistic(Z3)                  # logistica en la capa de salida
    return Z1, A1, Z2, A2, Z3, A3

Precisión No entrenada

In [9]:
# Calculo de precisión
test_predictions = forward_prop(X_test.transpose())[5]  # obtengo A3
test_comparisons = np.equal((test_predictions >= .5).flatten().astype(int), Y_test) # comparo predicciones con reales
accuracy = sum(test_comparisons.astype(int) / X_test.shape[0]) # calculo precisión

print(accuracy)

0.4477422335142317


### Back Propagation

In [10]:
# Tasa de aprendizaje
L = 0.01

# Derivadas de las funciones de activación
d_relu = lambda x: x > 0
d_logistic = lambda x: np.exp(-x) / (1 + np.exp(-x)) ** 2

# Función de backward propagation
# calculos con derivadas para obtener la derivada del costo con respecto a W3, W2, W1, B3, B2 B1
def backward_prop(Z1, A1, Z2, A2, Z3, A3, X, Y):
    dC_dA3 = 2 * (A3 - Y)           
    dA3_dZ3 = d_logistic(Z3)
    dZ3_dW3 = A2
    dZ3_dA2 = w_output
    dC_dZ3 = dC_dA3 * dA3_dZ3   # lo que me faltaba en mi codigo anterior para los proximos calculos 
                                # * para realizar la multiplicaion elemento por elemento 
                                # multiplico dos gradientes para obtener como cambia el costo cuando cambia la entrada ponderada (otro gradiente) 

    dC_dA2 = dZ3_dA2.T @ dC_dZ3
    dA2_dZ2 = d_relu(Z2)
    dZ2_dW2 = A1
    dZ2_dA1 = w_hidden2
    dC_dZ2 = dC_dA2 * dA2_dZ2

    dC_dA1 = dZ2_dA1.T @ dC_dZ2
    dA1_dZ1 = d_relu(Z1)
    dZ1_dW1 = X
    dC_dZ1 = dC_dA1 * dA1_dZ1

    dC_dW3 = dC_dZ3 @ dZ3_dW3.T
    # dC_dB3 (almacena suma) gradiente total de la función de costo con respecto al sesgo
    # np.sum() # suma los gradientes para cada muestra de entrenamiento
    # dC_dZ3 como cambia el costo cuando cambia la entrada 
    # axis = 1 a lo largo del eje 1 (columnas)
    # keepdims mantiene la dimension original
    dC_dB3 = np.sum(dC_dZ3, axis=1, keepdims=True)  

    dC_dW2 = dC_dZ2 @ dZ2_dW2.T
    dC_dB2 = np.sum(dC_dZ2, axis=1, keepdims=True)

    dC_dW1 = dC_dZ1 @ dZ1_dW1.T
    dC_dB1 = np.sum(dC_dZ1, axis=1, keepdims=True)
    
    return dC_dW1, dC_dB1, dC_dW2, dC_dB2, dC_dW3, dC_dB3



#### Descenso de Gradiente Estocástico

In [13]:
for i in range(150_000):
    # seleccionar aleatoriamente uno de los datos de entrenamiento
    idx = np.random.choice(n, 1, replace=False)
    X_sample = X_train[idx].transpose()
    Y_sample = Y_train[idx]

    # pasar datos seleccionados aleatoriamente a través de la red neuronal
    Z1, A1, Z2, A2, Z3, A3 = forward_prop(X_sample)

    # distribuir error a través de la retropropagación
    # y devolver pendientes para pesos y sesgos
    dW1, dB1, dW2, dB2, dW3, dB3 = backward_prop(Z1, A1, Z2, A2, Z3, A3, X_sample, Y_sample)

    # actualizar pesos y sesgos
    w_hidden -= L * dW1
    b_hidden -= L * dB1
    w_hidden2 -= L * dW2
    b_hidden2 -= L * dB2
    w_output -= L * dW3
    b_output -= L * dB3

Precisión Red Entrenada

In [14]:
# Calculo de precisión de entrenamiento
train_predictions = forward_prop(X_train.transpose())[5]  # Obtener A3 para el conjunto de entrenamiento
train_comparisons = np.equal((train_predictions >= .5).flatten().astype(int), Y_train)
train_accuracy = sum(train_comparisons.astype(int)) / X_train.shape[0]

print(f"Train Accuracy: {train_accuracy}")
    
    
# Calculo de precisión de prueba
test_predictions = forward_prop(X_test.transpose())[5]  # Obtener A3 para el conjunto de prueba
test_comparisons = np.equal((test_predictions >= .5).flatten().astype(int), Y_test)
test_accuracy = sum(test_comparisons.astype(int)) / X_test.shape[0]

print(f"Test Accuracy: {test_accuracy}")


Train Accuracy: 0.928567303805509
Test Accuracy: 0.9238942141124841


### Probando la Red

In [17]:
import numpy as np
import pandas as pd

# Diccionarios definidos previamente
cus_type = {'Loyal Customer': 0, 'disloyal Customer': 1}
travel_type = {'Business travel': 0, 'Personal Travel': 1}
clasS = {'Business': 2, 'Eco': 1, 'Eco Plus': 0}
flight_dist = {(31, 650): 0, (650, 1269): 1, (1269, 1888): 2, (1888, 2507): 3,
               (2507, 3126): 4, (3126, 3745): 5, (3745, 4364): 6, (4364, 4984): 7}

# Función para pedir datos al usuario
def pedir_datos():
    customer_type = input("Customer Type (Loyal Customer, disloyal Customer): ")
    type_of_travel = input("Type of Travel (Business travel, Personal Travel): ")
    clasS_input = input("Class (Business, Eco, Eco Plus): ")
    flight_distance = int(input("Flight Distance (31-4984): "))
    
    inflight_wifi = int(input("Inflight wifi service (0-5): "))
    online_booking = int(input("Ease of Online booking (0-5): "))
    food_drink = int(input("Food and drink (0-5): "))
    online_boarding = int(input("Online boarding (0-5): "))
    seat_comfort = int(input("Seat comfort (0-5): "))
    inflight_entertainment = int(input("Inflight entertainment (0-5): "))
    onboard_service = int(input("On-board service (0-5): "))
    legroom_service = int(input("Leg room service (0-5): "))
    baggage_handling = int(input("Baggage handling (0-5): "))
    checkin_service = int(input("Checkin service (0-5): "))
    inflight_service = int(input("Inflight service (0-5): "))
    cleanliness = int(input("Cleanliness (0-5): "))
    
    return {
        'Customer Type': cus_type[customer_type],
        'Type of Travel': travel_type[type_of_travel],
        'Class': clasS[clasS_input],
        'Flight Distance': get_flight_distance(flight_distance),
        'Inflight wifi service': inflight_wifi,
        'Ease of Online booking': online_booking,
        'Food and drink': food_drink,
        'Online boarding': online_boarding,
        'Seat comfort': seat_comfort,
        'Inflight entertainment': inflight_entertainment,
        'On-board service': onboard_service,
        'Leg room service': legroom_service,
        'Baggage handling': baggage_handling,
        'Checkin service': checkin_service,
        'Inflight service': inflight_service,
        'Cleanliness': cleanliness
    }

# Función para convertir la distancia de vuelo al rango correspondiente
def get_flight_distance(distance):
    for key, value in flight_dist.items():
        if key[0] <= distance < key[1]:
            return value
    return flight_dist[(4364, 4984)]

# Solicitar al usuario que ingrese los datos
datos_usuario = pedir_datos()

# Convertir los datos a un DataFrame de Pandas
new_data = pd.DataFrame([datos_usuario])

# Escalar los datos de entrada
new_inputs = scaler.transform(new_data.values)
Z1, A1, Z2, A2, Z3, A3 = forward_prop(new_inputs.transpose())

# Definir una función para clasificar la satisfacción
def clasificar_satisfaccion(valor):
    if valor >= 0.5:
        return "satisfied"
    else:
        return "neutral or dissatisfied"

# Clasificar la satisfacción para cada dato de entrada
satisfaccion_estimada = [clasificar_satisfaccion(valor) for valor in A3[0]]

# Agregar la satisfacción predicha al DataFrame
new_data['Satisfaction'] = satisfaccion_estimada

# Imprimir los datos con la satisfacción predicha
new_data


,Customer Type,Type of Travel,Class,Flight Distance,Inflight wifi service,Ease of Online booking,Food and drink,Online boarding,Seat comfort,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Satisfaction
0,0,0,1,0,5,4,5,4,3,5,4,5,3,5,4,5,satisfied


### Scikit-Learn

In [413]:
import pandas as pd
# cargar datos
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier

df = pd.read_csv('pasajeros_actualizado.csv')

# Extraer variables de entrada (todas las filas, todas las columnas menos la última)
# Nota que deberíamos hacer algún escalado lineal aquí
X = (df.values[:, :-1])

# Extraer columna de salida (todas las filas, última columna)
Y = df.values[:, -1]

# Separar los datos de entrenamiento y prueba
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=1/3)

nn = MLPClassifier(solver='adam',
                   hidden_layer_sizes=(12, 8, ),
                   activation='relu',
                   max_iter=150_000,
                   learning_rate_init=.01)

nn.fit(X_train, Y_train)

print("Puntaje del conjunto de entrenamiento: %f" % nn.score(X_train, Y_train))
print("Puntaje del conjunto de prueba: %f" % nn.score(X_test, Y_test))


Puntaje del conjunto de entrenamiento: 0.948317
Puntaje del conjunto de prueba: 0.944335
